In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pyreadstat
import matplotlib.pyplot as plt

DATABASE = "database\\"
MERGED_DF = {}

In [2]:
os.listdir("database\\")

['1999-2000',
 '2001-2002',
 '2003-2004',
 '2005-2006',
 '2007-2008',
 '2009-2010',
 '2011-2012',
 'report.docx']

In [3]:
def get_DF_of_year(year, threshold=5):
    result = {}
    db = glob.glob(f"{DATABASE}{year}\\Lab\\*")
    for xpt in db:
        df, meta = pyreadstat.read_xport(xpt)
        df_new = df.dropna(thresh=int(len(df.columns)-(threshold/100)*len(df.columns)))
        df_new["year"] = year
        xpt_name = xpt.split("\\")[-1]
        result[xpt_name] = df_new
        print(f"{xpt_name} : {len(df_new)/len(df)} : with at most {threshold}% of NAN")
    return result

In [12]:
year_1999 = get_DF_of_year(year="1999-2000", threshold=2)
year_2001 = get_DF_of_year(year="2001-2002", threshold=2)
year_2003 = get_DF_of_year(year="2003-2004", threshold=2)

LAB10.xpt : 1.0 : with at most 2% of NAN
LAB10AM.xpt : 0.911233547597184 : with at most 2% of NAN
LAB11.xpt : 0.3184324065196548 : with at most 2% of NAN


<ipython-input-3-6b673aaee645>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["year"] = year


LAB13.xpt : 0.8895014381591563 : with at most 2% of NAN
LAB13AM.xpt : 0.8099616858237548 : with at most 2% of NAN
LAB16.xpt : 0.972758664790894 : with at most 2% of NAN
SSCARD_A.xpt : 0.9943809958885336 : with at most 2% of NAN
SSCYST_A.xpt : 1.0 : with at most 2% of NAN
L10AM_B.xpt : 0.9206219312602292 : with at most 2% of NAN
L10_2_B.xpt : 0.36804308797127466 : with at most 2% of NAN
L10_B.xpt : 1.0 : with at most 2% of NAN
L11_2_B.xpt : 0.466786355475763 : with at most 2% of NAN
L11_B.xpt : 0.3229324120060462 : with at most 2% of NAN
L13AM_B.xpt : 0.816084229390681 : with at most 2% of NAN
L13_2_B.xpt : 0.5073529411764706 : with at most 2% of NAN
L13_B.xpt : 0.8947311595767653 : with at most 2% of NAN
L16_B.xpt : 0.9683127096102695 : with at most 2% of NAN
SSCARD_A.xpt : 0.9943809958885336 : with at most 2% of NAN
SSCYST_B.xpt : 1.0 : with at most 2% of NAN
L10AM_C.xpt : 0.933253873659118 : with at most 2% of NAN
L10_C.xpt : 1.0 : with at most 2% of NAN
L11_C.xpt : 0.800522932781348

### 1) LAB10

In [13]:
df_lab10 = pd.concat([year_1999["LAB10.xpt"], 
                                year_2001["L10_B.xpt"], 
                                year_2003["L10_C.xpt"]],axis=0).reset_index(drop=True)
MERGED_DF["LAB10"] = df_lab10
MERGED_DF

{'LAB10':           SEQN  LBXGH       year
 0          2.0    4.7  1999-2000
 1          5.0    5.5  1999-2000
 2          6.0    5.1  1999-2000
 3          7.0    5.8  1999-2000
 4          8.0    5.0  1999-2000
 ...        ...    ...        ...
 21188  31119.0    5.6  2003-2004
 21189  31121.0    5.9  2003-2004
 21190  31123.0    5.3  2003-2004
 21191  31124.0    5.6  2003-2004
 21192  31125.0    5.3  2003-2004
 
 [21193 rows x 3 columns],
 'L10AM':          SEQN       WTSAF2YR     LBXGLU  LBXGLUSI  LBXCPSI  LBXIN  LBXINSI  \
 0         2.0   60586.147294   81.63259     4.646    0.484   4.55    27.30   
 1         5.0  234895.205650   97.87633     5.550    0.951  13.65    81.90   
 2         7.0   57661.621988   83.53772     4.756    0.546   9.72    58.32   
 3         8.0   76026.438279   82.13394     4.678    0.337   7.63    45.78   
 4        10.0   60202.416895   87.74906     4.989    0.517   6.43    38.58   
 ...       ...            ...        ...       ...      ...    ...     

### 2) LAB10AM

In [15]:
year_2003["L10AM_C.xpt"]

,SEQN,WTSAF2YR,LBXGLU,LBDGLUSI,LBXCPSI,LBXIN,LBDINSI,year
0,21005.0,14084.10,87.6,4.863,0.591,11.26,67.56,2003-2004
1,21006.0,13080.94,75.0,4.163,0.535,11.44,68.64,2003-2004
2,21013.0,17173.77,91.7,5.090,0.593,10.63,63.78,2003-2004
3,21016.0,17008.13,79.3,4.402,0.619,16.45,98.70,2003-2004
4,21020.0,65607.89,88.1,4.890,0.717,9.05,54.30,2003-2004
...,...,...,...,...,...,...,...,...
3351,31114.0,104950.92,92.6,5.140,0.384,2.59,15.54,2003-2004
3352,31115.0,10521.93,82.2,4.563,0.740,10.26,61.56,2003-2004
3353,31116.0,170326.69,80.5,4.469,0.403,2.24,13.44,2003-2004
3354,31119.0,70025.16,111.7,6.200,2.003,40.42,242.52,2003-2004


In [16]:
year_1999['LAB10AM.xpt'].columns

Index(['SEQN', 'WTSAF2YR', 'WTSAF4YR', 'LBXGLU', 'LBXGLUSI', 'LBXCPSI',
       'LBXIN', 'LBXINSI', 'year'],
      dtype='object')

In [17]:
def get_Tosoh_equivalent(x):
    return 1.0027 * x - 2.2934

In [18]:
year_1999['LAB10AM.xpt']["LBXGLU"] = get_Tosoh_equivalent(year_1999['LAB10AM.xpt']["LBXGLU"])
year_2001['L10AM_B.xpt']["LBXGLU"] = get_Tosoh_equivalent(year_2001['L10AM_B.xpt']["LBXGLU"])
year_2003['L10AM_C.xpt']["LBXGLU"] = year_2003['L10AM_C.xpt']["LBXGLU"] + 0.05

In [19]:
year_1999['LAB10AM.xpt']["WTSAF2YR"] = year_1999['LAB10AM.xpt']["WTSAF4YR"]/2
year_2001['L10AM_B.xpt']["WTSAF2YR"] = year_2001['L10AM_B.xpt']["WTSAF4YR"]/2

year_1999['LAB10AM.xpt'].drop(columns=["WTSAF4YR"], inplace=True)
year_2001['L10AM_B.xpt'].drop(columns=["WTSAF4YR"], inplace=True)

In [27]:
MERGED_DF["L10AM"] = pd.concat([year_1999['LAB10AM.xpt'], 
                                year_2001['L10AM_B.xpt'], 
                                year_2003['L10AM_C.xpt']],
                              axis=0)

In [28]:
#MERGED_DF["L10AM"].drop(columns=["LBXINSI", 
 #                                "LBDGLUSI", 
  #                               "LBDINSI"], 
   #                     inplace=True)

In [29]:
MERGED_DF["L10AM"].reset_index(drop=True, inplace=True)

In [31]:
MERGED_DF["L10AM"]

,SEQN,WTSAF2YR,LBXGLU,LBXGLUSI,LBXCPSI,LBXIN,LBXINSI,year,LBDGLUSI,LBDINSI
0,2.0,16536.633786,81.63259,4.646,0.484,4.55,27.30,1999-2000,NaN,NaN
1,5.0,49234.403246,97.87633,5.550,0.951,13.65,81.90,1999-2000,NaN,NaN
2,7.0,16467.937032,83.53772,4.756,0.546,9.72,58.32,1999-2000,NaN,NaN
3,8.0,18008.383815,82.13394,4.678,0.337,7.63,45.78,1999-2000,NaN,NaN
4,10.0,14469.761221,87.74906,4.989,0.517,6.43,38.58,1999-2000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9479,31114.0,104950.920000,92.60000,NaN,0.384,2.59,NaN,2003-2004,5.140,15.54
9480,31115.0,10521.930000,82.20000,NaN,0.740,10.26,NaN,2003-2004,4.563,61.56
9481,31116.0,170326.690000,80.50000,NaN,0.403,2.24,NaN,2003-2004,4.469,13.44
9482,31119.0,70025.160000,111.70000,NaN,2.003,40.42,NaN,2003-2004,6.200,242.52


### 3) LAB11

In [36]:
year_1999["LAB11.xpt"]
year_2001["L11_2_B.xpt"]
year_2003["L11_C.xpt"]

,SEQN,LBXCRP,LBXBAP,LBXPT21,year
0,21005.0,1.10,16.4,56.0,2003-2004
1,21006.0,0.07,20.7,47.0,2003-2004
2,21007.0,0.02,15.9,40.0,2003-2004
3,21008.0,0.01,89.5,37.0,2003-2004
4,21009.0,0.08,NaN,35.0,2003-2004
...,...,...,...,...,...
9172,31119.0,0.12,23.7,24.0,2003-2004
9173,31121.0,1.27,NaN,61.0,2003-2004
9175,31123.0,0.20,NaN,63.0,2003-2004
9176,31124.0,0.16,19.2,35.0,2003-2004


In [37]:
year_2001["L11_2_B.xpt"].rename(columns={"LB2CRP":"LBXCRP"},
                               inplace=True)

In [38]:
year_1999["LAB11.xpt"].drop(columns=["LBXFB",
                                     "LBDFBSI",
                                     "URXNT"],
                           inplace=True)
year_2001["L11_2_B.xpt"].drop(columns=["LB2FB", 
                                      "LB2FBSI",
                                      "UR2NT"],
                             inplace=True)

In [ ]:
# LB2BAP=exp(−0.5326+1.1139×log(LBXBAP)−0.7963×max(0,log(LBXBAP)−4.5151)+0.9660×max(0,log(LBXBAP)−4.9030))

In [45]:
def convert_to_BeckmanAccess(x):
    log = np.log(x)
    return np.exp((((-0.5326)*(1.1139*log))-
                   (- 0.7963 * np.maximum(0, log - 4.5151))+
                   (0.9660 * np.maximum(0, log - 4.9030))))

In [39]:
year_2001["L11_2_B.xpt"].rename(columns={"LB2BAP":"LBXBAP"},
                               inplace=True)

In [46]:
year_2001["L11_2_B.xpt"]["LBXBAP"] = convert_to_BeckmanAccess(year_2001["L11_2_B.xpt"]["LBXBAP"])
year_1999["LAB11.xpt"]["LBXBAP"] = convert_to_BeckmanAccess(year_1999["LAB11.xpt"]["LBXBAP"])

### 4) Cholesterol (Total & HDL) - LAB13

In [ ]:
LBXHDD → LBDHDL
LBDHDDSI → LBDHDLSI

In [50]:
year_2003["L13_C.xpt"].rename(columns={"LBXHDD":"LBDHDL",
                                      "LBDHDDSI":"LBDHDLSI"},
                             inplace=True)

In [53]:
year_1999["LAB13.xpt"]["LBDHDLSI"] = year_1999["LAB13.xpt"]["LBDHDL"] * 0.02586
year_2001["L13_B.xpt"]["LBDHDLSI"] = year_2001["L13_B.xpt"]["LBDHDL"] * 0.02586

### 5) LDL-Cholesterol & Triglycerides - LAB13AM